# 25/12/03 FANS count data Modelling and inference
In the previous notebook, I made a variety of exploratory plots to get a sense of which predictor variables may correlate with the nuclei counts. Both technical predictors, such as `sample_mass_mg`, `date_nuc_prep` and `incubation_time_hrs`, are included, as well as the three biological predictors of interest- `dpi`, `inoculum`, and `population`. 

In this notebook we will build a model for this data, and use its predctions as input to formal statistical tests.
## Section 1: Formal model selection
Here, I show formally that the negative binomial model is a better fit for my data than the Poisson. This is expected given the nature of the dataset, and going through these steps is probably overkill. A key difference between them is that the NB model accounts for overdispersion, whereas the Pois model assumes the variance and mean are approximately equal
1. First, by calculating the variance: mean ratio
2. Via a dispersion test after fitting an intercept-only Poisson model, we are assessing whether the observed variability in our data is compatible with a Poisson distribution whose mean equals the sample mean.
3. Fitting both an NB and a Poisson model and comparing AIC.


In [26]:
from pathlib import Path
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from plotnine import *
from plotnine import scales
from marginaleffects import *

In [2]:
# load in the tidied, long-format FANS data
base_path = Path("/home/tmurphy/phd_work/experiment_383/exp_383_flowCytometry_analysis")
df_path = Path(base_path / "tidied_FANS_data/exp383_tidy_FANS_data_long.csv")
df = pd.read_csv(df_path)

# we only want the raw count data for this analysis.
df_raw = df[df['count_statistic']=='EstTotalCount']
# remind ourselves of the data
df_raw

,animal_id,group_no,inoculum,inoculation_batch,dpi,sample_mass_mg,date_nuc_prep,incubation_time_hrs,population,count_statistic,count_value
0,918310,1,RML,BATCH4,60,246,2025-05-06,19.00,NeuN+,EstTotalCount,4359464.0
1,917423,19,22L,BATCH3,120,207,2025-05-06,19.67,NeuN+,EstTotalCount,2913658.0
2,918309,4,CBH,BATCH4,60,240,2025-05-07,16.92,NeuN+,EstTotalCount,5017422.0
3,916462,1,RML,BATCH1,60,248,2025-05-08,18.50,NeuN+,EstTotalCount,4324274.0
4,918277,9,RML,BATCH4,90,220,2025-05-08,18.98,NeuN+,EstTotalCount,5102254.0
...,...,...,...,...,...,...,...,...,...,...,...
283,918318,4,CBH,BATCH4,60,221,2025-08-19,19.48,SOX2+,EstTotalCount,152305.0
284,917440,4,CBH,BATCH3,60,254,2025-08-20,18.80,SOX2+,EstTotalCount,261952.0
285,916468,19,22L,BATCH1,120,243,2025-08-20,19.43,SOX2+,EstTotalCount,171849.0
286,917445,3,22L,BATCH3,60,294,2025-08-21,22.62,SOX2+,EstTotalCount,358748.0


In [3]:
# Assess the ratio of the variance to the mean
df_meanVar = (
    df_raw.groupby(['dpi', 'inoculum', 'population'])
    .agg(
        mean=('count_value', 'mean'),
        variance=('count_value', 'var')
    )
)
df_meanVar['var/mean'] = df_meanVar['variance'] / df_meanVar['mean']
df_meanVar.head() # variance >> mean

mean      variance       var/mean
dpi inoculum population                                           
60  22L      NeuN+       5.296955e+06  1.317430e+12  248714.543710
             PU1+        2.039002e+05  4.433085e+09   21741.449272
             SOX10+      1.014694e+06  1.225343e+11  120759.788091
             SOX2+       3.042200e+05  1.213848e+10   39900.343298
    CBH      NeuN+       4.468945e+06  1.469023e+12  328717.978484

In [4]:
# Formal test of Poisson goodness of fit
# Fit minimal Poisson model (intercept only for raw dispersion test)

# Fit a minimal Poisson model with only an intercept term
m_pois = smf.glm(
    "count_value ~ 1",
    data=df_raw,
    family=sm.families.Poisson()
).fit()

resid_deviance = m_pois.deviance
resid_df       = m_pois.df_resid
dispersion     = resid_deviance / resid_df

print("Residual deviance:", resid_deviance)
print("Residual df:", resid_df)
print("Dispersion:", dispersion)

Residual deviance: 611148138.3893344
Residual df: 287
Dispersion: 2129436.0222624894


In [5]:
# Compare Poisson and NB goodness of fit
# Poisson model

# NB model with intercept only (no predictors)
m_nb_basic = smf.glm(
    "count_value ~ 1",
    data=df_raw,
    family=sm.families.NegativeBinomial()
).fit()

# Poisson model with biological predictors
m_pois_biol = smf.glm(
    "count_value ~ dpi + inoculum + population",
    data=df_raw,
    family=sm.families.Poisson()
).fit()

# NB model with biological predictors
m_nb_biol = smf.glm(
    "count_value ~ dpi + inoculum + population",
    data=df_raw,
    family=sm.families.NegativeBinomial()
).fit()

# Insepct AICs
print("Poisson (Intercept only) AIC:", m_pois.aic)
print("Poisson (+Predictors) AIC:", m_pois_biol.aic)
print("NB AIC:     ", m_nb_basic.aic)
print("NB (+Predictors):", m_nb_biol.aic)

Poisson (Intercept only) AIC: 611152541.0597963
Poisson (+Predictors) AIC: 29518120.822409708
NB AIC:      8792.233545914758
NB (+Predictors): 8360.556103685405


/home/tmurphy/miniforge3/envs/generalPython/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
/home/tmurphy/miniforge3/envs/generalPython/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.


In [61]:
# Building the NB model upwards from technical predictor variables

# Start with a model that only includes mass and an offset (scaling-factor)
# We expect that counts
m_nb_0 = smf.glm(
    "count_value ~ 1",
    data=df_raw,
    family=sm.families.NegativeBinomial(),
    offset=np.log(df_raw['sample_mass_mg'])
).fit()

m_nb_0.summary2()

/home/tmurphy/miniforge3/envs/generalPython/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.


<class 'statsmodels.iolib.summary2.Summary'>
"""
               Results: Generalized linear model
===============================================================
Model:              GLM              AIC:            8790.2847 
Link Function:      Log              BIC:            -1155.7092
Dependent Variable: count_value      Log-Likelihood: -4394.1   
Date:               2025-12-03 16:33 LL-Null:        -4394.1   
No. Observations:   288              Deviance:       469.56    
Df Model:           0                Pearson chi2:   464.      
Df Residuals:       287              Scale:          1.0000    
Method:             IRLS                                       
----------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|   [0.025  0.975]
----------------------------------------------------------------
Intercept     8.7956    0.0589  149.2670  0.0000  8.6802  8.9111
===============================================================

"""

In [62]:
# Add incubation time as a linear predictor
m_nb_1 = smf.glm(
    "count_value ~ incubation_time_hrs",
    data=df_raw,
    family=sm.families.NegativeBinomial(),
    offset=np.log(df_raw['sample_mass_mg'])    
).fit()

m_nb_1.summary2()

/home/tmurphy/miniforge3/envs/generalPython/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.


<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Generalized linear model
===================================================================
Model:                GLM               AIC:             8792.2543 
Link Function:        Log               BIC:             -1150.0766
Dependent Variable:   count_value       Log-Likelihood:  -4394.1   
Date:                 2025-12-03 16:34  LL-Null:         -4394.1   
No. Observations:     288               Deviance:        469.53    
Df Model:             1                 Pearson chi2:    464.      
Df Residuals:         286               Scale:           1.0000    
Method:               IRLS                                         
-------------------------------------------------------------------
                     Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-------------------------------------------------------------------
Intercept            8.9877   1.1266  7.9779 0.0000  6.7796 11.1957
incubation_time_hrs -0.0103   0.0605 -0.1707 0.8644 -0.1289  0.1082
===================================================================

"""

In [60]:
# Incubation time may only matter for certain populations
# of nuclei
# Build a model with an interation term
m_nb_2 = smf.glm(
    "count_value ~ incubation_time_hrs * population",
    data=df_raw,
    family=sm.families.NegativeBinomial(),
    offset=np.log(df_raw["sample_mass_mg"])
).fit()

m_nb_2.summary2()

/home/tmurphy/miniforge3/envs/generalPython/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.


<class 'statsmodels.iolib.summary2.Summary'>
"""
                           Results: Generalized linear model
========================================================================================
Model:                       GLM                      AIC:                    8360.9157 
Link Function:               Log                      BIC:                    -1559.4375
Dependent Variable:          count_value              Log-Likelihood:         -4172.5   
Date:                        2025-12-03 16:31         LL-Null:                -4394.1   
No. Observations:            288                      Deviance:               26.191    
Df Model:                    7                        Pearson chi2:           26.1      
Df Residuals:                280                      Scale:                  1.0000    
Method:                      IRLS                                                       
----------------------------------------------------------------------------------------
                                          Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
----------------------------------------------------------------------------------------
Intercept                                10.0078   2.2531  4.4417 0.0000  5.5917 14.4238
population[T.PU1+]                       -2.2578   3.1864 -0.7086 0.4786 -8.5030  3.9875
population[T.SOX10+]                     -1.2644   3.1864 -0.3968 0.6915 -7.5096  4.9808
population[T.SOX2+]                      -2.3917   3.1864 -0.7506 0.4529 -8.6369  3.8536
incubation_time_hrs                      -0.0047   0.1210 -0.0388 0.9690 -0.2418  0.2324
incubation_time_hrs:population[T.PU1+]   -0.0422   0.1711 -0.2467 0.8051 -0.3775  0.2931
incubation_time_hrs:population[T.SOX10+] -0.0195   0.1711 -0.1141 0.9092 -0.3548  0.3158
incubation_time_hrs:population[T.SOX2+]  -0.0278   0.1711 -0.1622 0.8711 -0.3631  0.3075
========================================================================================

"""

In [63]:
# The EDA suggests nuclei counts increased over the months 
# spent in the lab (date_nuc_prep), so we need to model this.

# Recode the date_nuc_prep as an ordinal time variable
# Avoids treating datetime as a categorical variable with loads of levels.

# 1. To datetime
df_raw["date_nuc_prep"] = pd.to_datetime(df_raw["date_nuc_prep"])

# 2. datetime --> ordinal (as new col)
df_raw['date_ord'] = df_raw['date_nuc_prep'].dt.date.apply(lambda d: d.toordinal())

# Fit the model (date of prep as ordinal)
m_nb_date = smf.glm(
    "count_value ~ date_ord",
    data=df_raw,
    family=sm.families.NegativeBinomial(),
    offset=np.log(df_raw["sample_mass_mg"])
).fit()

m_nb_date.summary2()

/tmp/ipykernel_3205475/338466913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_3205475/338466913.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/tmurphy/miniforge3/envs/generalPython/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.


<class 'statsmodels.iolib.summary2.Summary'>
"""
               Results: Generalized linear model
================================================================
Model:               GLM              AIC:            8788.6309 
Link Function:       Log              BIC:            -1153.7001
Dependent Variable:  count_value      Log-Likelihood: -4392.3   
Date:                2025-12-03 16:35 LL-Null:        -4394.1   
No. Observations:    288              Deviance:       465.91    
Df Model:            1                Pearson chi2:   456.      
Df Residuals:        286              Scale:          1.0000    
Method:              IRLS                                       
----------------------------------------------------------------
            Coef.     Std.Err.    z    P>|z|    [0.025    0.975]
----------------------------------------------------------------
Intercept -2585.4508 1314.9781 -1.9662 0.0493 -5162.7605 -8.1412
date_ord      0.0035    0.0018  1.9728 0.0485     0.0000  0.0070
================================================================

"""

In [64]:
# Model that factors into account an interaction between
# the date of the prep and the population
# Rational: upward trend in nuclei yield as time went on, when the antibodies 
# were not problematic  (i.e. NeuN), but there is no obvious link between date and yield
# for populations where antibody issues occurred
m_nb_dateXpop = smf.glm(
    "count_value ~ date_ord * population",
    data=df_raw,
    family=sm.families.NegativeBinomial(),
    offset=np.log(df_raw['sample_mass_mg'])
).fit()

m_nb_dateXpop.summary2()

/home/tmurphy/miniforge3/envs/generalPython/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.


<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Generalized linear model
======================================================================================
Model:                       GLM                     AIC:                   8358.0575 
Link Function:               Log                     BIC:                   -1562.2957
Dependent Variable:          count_value             Log-Likelihood:        -4171.0   
Date:                        2025-12-03 16:35        LL-Null:               -4394.1   
No. Observations:            288                     Deviance:              23.333    
Df Model:                    7                       Pearson chi2:          23.8      
Df Residuals:                280                     Scale:                 1.0000    
Method:                      IRLS                                                     
--------------------------------------------------------------------------------------
                                Coef.     Std.Err.    z    P>|z|    [0.025     0.975] 
--------------------------------------------------------------------------------------
Intercept                     -2636.3199 2629.9529 -1.0024 0.3161 -7790.9328 2518.2931
population[T.PU1+]             -164.6035 3719.3226 -0.0443 0.9647 -7454.3418 7125.1348
population[T.SOX10+]             17.4112 3719.3196  0.0047 0.9963 -7272.3213 7307.1437
population[T.SOX2+]            1372.9620 3719.3221  0.3691 0.7120 -5916.7754 8662.6994
date_ord                          0.0036    0.0036  1.0062 0.3143    -0.0034    0.0105
date_ord:population[T.PU1+]       0.0002    0.0050  0.0434 0.9654    -0.0096    0.0101
date_ord:population[T.SOX10+]    -0.0000    0.0050 -0.0051 0.9959    -0.0099    0.0098
date_ord:population[T.SOX2+]     -0.0019    0.0050 -0.3699 0.7114    -0.0117    0.0080
======================================================================================

"""

In [65]:
# Take a look at the AIC values for the technical predictor models we made
df_aic = pd.DataFrame(
    {
        'model': ["Mass_offset_model", "Incubation_model", "Incubation*pop_model", "PrepDate_model", "Date*pop_model"],
        'AIC': [m_nb_0.aic, m_nb_1.aic, m_nb_2.aic, m_nb_date.aic, m_nb_dateXpop.aic]
    }
).sort_values(by='AIC').reset_index(drop=True)

df_aic # Date*pop_model and incubation*pop_model are very close. 

,model,AIC
0,Date*pop_model,8358.057523
1,Incubation*pop_model,8360.915704
2,PrepDate_model,8788.630882
3,Mass_offset_model,8790.284709
4,Incubation_model,8792.254299


In [66]:
# fit another glm that examines the interactions between incubation,
# date and population.

m_nb_datePopIncub = smf.glm(
    "count_value ~  incubation_time_hrs * population + date_ord * population",
    data=df_raw,
    family=sm.families.NegativeBinomial(),
    offset=np.log(df_raw['sample_mass_mg'])
).fit()

m_nb_datePopIncub.summary2() 

/home/tmurphy/miniforge3/envs/generalPython/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                Results: Generalized linear model
=================================================================================================
Model:                          GLM                         AIC:                       8365.6854 
Link Function:                  Log                         BIC:                       -1540.0159
Dependent Variable:             count_value                 Log-Likelihood:            -4170.8   
Date:                           2025-12-03 17:04            LL-Null:                   -4394.1   
No. Observations:               288                         Deviance:                  22.961    
Df Model:                       11                          Pearson chi2:              23.4      
Df Residuals:                   276                         Scale:                     1.0000    
Method:                         IRLS                                                             
-------------------------------------------------------------------------------------------------
                                           Coef.     Std.Err.    z    P>|z|    [0.025     0.975] 
-------------------------------------------------------------------------------------------------
Intercept                                -2668.8974 2646.7485 -1.0084 0.3133 -7856.4291 2518.6344
population[T.PU1+]                        -134.0520 3743.0730 -0.0358 0.9714 -7470.3403 7202.2364
population[T.SOX10+]                       -24.4698 3743.0700 -0.0065 0.9948 -7360.7523 7311.8127
population[T.SOX2+]                       1307.7063 3743.0726  0.3494 0.7268 -6028.5812 8643.9937
incubation_time_hrs                         -0.0162    0.1218 -0.1332 0.8940    -0.2548    0.2224
incubation_time_hrs:population[T.PU1+]      -0.0334    0.1722 -0.1941 0.8461    -0.3709    0.3041
incubation_time_hrs:population[T.SOX10+]    -0.0199    0.1722 -0.1153 0.9082    -0.3573    0.3176
incubation_time_hrs:population[T.SOX2+]     -0.0232    0.1722 -0.1346 0.8929    -0.3606    0.3143
date_ord                                     0.0036    0.0036  1.0121 0.3115    -0.0034    0.0106
date_ord:population[T.PU1+]                  0.0002    0.0051  0.0352 0.9719    -0.0097    0.0101
date_ord:population[T.SOX10+]                0.0000    0.0051  0.0062 0.9951    -0.0099    0.0100
date_ord:population[T.SOX2+]                -0.0018    0.0051 -0.3500 0.7263    -0.0117    0.0082
=================================================================================================

"""

In [72]:
# Check for collinearity between incubation and date
prep_date_incubation_corr = (
    df_raw[['incubation_time_hrs', 'date_ord']].corr()
)
prep_date_incubation_corr

,incubation_time_hrs,date_ord
incubation_time_hrs,1.000000,0.113317
date_ord,0.113317,1.000000


In [73]:
# Check for collinearity between incubation and date
prep_date_incubation_corr = (
    df_raw.groupby('population')[['count_value', 'date_ord']].corr()
)
prep_date_incubation_corr

count_value  date_ord
population                                   
NeuN+      count_value     1.000000  0.487414
           date_ord        0.487414  1.000000
PU1+       count_value     1.000000  0.330710
           date_ord        0.330710  1.000000
SOX10+     count_value     1.000000  0.408281
           date_ord        0.408281  1.000000
SOX2+      count_value     1.000000  0.173158
           date_ord        0.173158  1.000000